In [50]:
import pysam
import pyfastx
import pandas as pd
from tqdm import tqdm
from Bio.Seq import Seq

an rev cmpl error is defined as:
- a probe sequence matches on the opposite strand of the targeted gene

first load the probe - gene relationships based on probe names:

In [13]:
probes = pyfastx.Fasta("/ccb/salz4-3/hji20/off-target-probe-checker/data/xenium_human_breast_gene_expression_panel_probe_sequences.fasta")

In [14]:
probe_targets = dict()
for probe in probes:
    temp = probe.name.split("|")
    gid = temp[0]
    gname = temp[1] # caution! the gene name may have changed or a synonym exists
    probe_targets[probe.name] = (gid, gname)

I want to be able to check the alignments to transcripts belonging to a given target gene, so I load GENCODE reference infos:

In [15]:
fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/gencode.v47.basic.annotation.fmted.gff"
df = pd.read_csv(fn, sep='\t', header=None)
df.columns = ['chr', 'src', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
df.head()

,chr,src,feature,start,end,score,strand,frame,attribute
0,chr1,HAVANA,gene,11121.0,24894.0,.,+,.,ID=ENSG00000290825.2;gene_id=ENSG00000290825.2...
1,chr1,HAVANA,transcript,11426.0,14409.0,.,+,.,ID=ENST00000832828.1;Parent=ENSG00000290825.2;...
2,chr1,HAVANA,exon,11426.0,11671.0,.,+,.,ID=exon:ENST00000832828.1:1;Parent=ENST0000083...
3,chr1,HAVANA,exon,12010.0,12227.0,.,+,.,ID=exon:ENST00000832828.1:2;Parent=ENST0000083...
4,chr1,HAVANA,exon,12613.0,12721.0,.,+,.,ID=exon:ENST00000832828.1:3;Parent=ENST0000083...


In [16]:
def get_tid(s): # with its parent
    temp = s.split(';')
    tid = None
    pid = None
    for x in temp:
        kv = x.split('=')
        if len(kv) != 2: continue
        if kv[0] == 'ID':
            tid = kv[1]
        elif kv[0] == 'Parent':
            pid = kv[1]
            break
    return tid, pid

ginfos = dict() # <k,v> = <pid, [tid]>
ctr = 0
for i, row in tqdm(df.iterrows()):
    if row['feature'] == 'transcript':
        tid, pid = get_tid(row['attribute'])
        ctr += 1
        if not tid or not pid:
            print("error parsing IDs")
            break
        if pid not in ginfos:
            ginfos[pid] = [tid]
        else:
            ginfos[pid].append(tid)
print(f'{ctr} transcripts loaded')

0it [00:00, ?it/s]

2223391it [01:15, 29490.33it/s]

158338 transcripts loaded


also create mapping from tid to gid for easy access:

In [17]:
t2g_tbl = dict()
for pid in ginfos:
    for tid in ginfos[pid]:
        t2g_tbl[tid] = pid

In [82]:
aln_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/results/bt2/xenium_human_breast_gene_expression_panel_probe_sequences.bam"

probe_aln_dirs = dict()
with pysam.AlignmentFile(aln_fn, 'rb') as fh:
    for brec in fh:
        if brec.is_unmapped: continue
        qname = brec.query_name
        target_gid, target_gname = probe_targets[qname]
        tname = brec.reference_name
        if qname not in probe_aln_dirs:
            probe_aln_dirs[qname] = []
        if target_gid == t2g_tbl[tname].split('.')[0]:
            probe_aln_dirs[qname].append(brec.is_forward)

In [83]:
probe_aln_dirs['ENSG00000184293|CLECL1|ad68b18']

[]

In [92]:
to_rc = []
for qname in probe_aln_dirs:
    if len(probe_aln_dirs[qname]) == 0:
        print(qname)
    elif all(probe_aln_dirs[qname]):
        continue
    else:
        assert all(not x for x in probe_aln_dirs[qname])
        to_rc.append(qname)
print(len(to_rc))

ENSG00000184293|CLECL1|ad68b18
ENSG00000184293|CLECL1|b96bdcf
ENSG00000184293|CLECL1|04513e0
ENSG00000184293|CLECL1|c5726c8
ENSG00000117090|SLAMF1|135aee1
ENSG00000128383|APOBEC3A|eba2db3
2508


In [85]:
'ENSG00000196616|ADH1B|07014f4' in to_rc

True

this is slightly more than expected based on counting the number of reads with only REV alignments against transcriptome, investigating where this discrepancy comes from:

In [86]:
amat = dict()
unmapped = []
with pysam.AlignmentFile(aln_fn, 'rb') as fh:
    for brec in fh:
        qname = brec.query_name
        if brec.is_unmapped: # highly likely to be intergenic
            unmapped.append(qname)
        if qname not in amat:
            amat[qname] = [brec.is_reverse]
        else:
            amat[qname].append(brec.is_reverse)

In [87]:
rev_only = []
for qname in amat:
    if False in amat[qname]:
        continue
    else:
        rev_only.append(qname)
print(f'{len(rev_only)} out of {len(probes)} reads with only REV alignments')

2412 out of 4809 reads with only REV alignments


In [88]:
amat['ENSG00000196628|TCF4|40e4db9']

[False, True, False, False, False]

In [89]:
for x in to_rc:
    if x not in rev_only:
        print(x)
        break

ENSG00000261371|PECAM1|639cb44


The number of wrongly oriented probe sequences is underestimated when the alignment records are used because accidentally there are FWD alignments to processed pseudogenes and etc.
- we'll also have to proceed with just the protein coding transcripts (see below)

In [24]:
def get_gtype(s):
    temp = s.split(';')
    gtype = None
    gid = None
    for x in temp:
        kv = x.split('=')
        if len(kv) != 2: continue
        if kv[0] == 'ID':
            gid = kv[1]
        elif kv[0] == 'gene_type':
            gtype = kv[1]
            break
    return gid, gtype

pc_genes = []
for i, row in tqdm(df.iterrows()):
    if row['feature'] == 'gene':
        gid, gtype = get_gtype(row['attribute'])
        if gtype == "protein_coding":
            pc_genes.append(gid)

2223391it [01:15, 29435.56it/s]


In [25]:
len(pc_genes)

20092

In [26]:
def get_ttype(s):
    temp = s.split(';')
    ttype = None
    tid = None
    pid = None
    for x in temp:
        kv = x.split('=')
        if len(kv) != 2: continue
        if kv[0] == 'ID':
            tid = kv[1]
        elif kv[0] == 'Parent':
            pid = kv[1]
        elif kv[0] == 'transcript_type':
            ttype = kv[1]
            break
    return tid, pid, ttype

pc_g2t = dict()
for i, row in tqdm(df.iterrows()):
    if row['feature'] == 'transcript':
        tid, pid, ttype = get_ttype(row['attribute'])
        if ttype == "protein_coding":
            assert pid in pc_genes
            if pid not in pc_g2t:
                pc_g2t[pid] = [tid]
            else:
                pc_g2t[pid].append(tid)

2223391it [01:24, 26182.56it/s]


In [27]:
for pid in pc_genes:
    if pid not in pc_g2t:
        print(pid)

ENSG00000285629.1
ENSG00000211454.16
ENSG00000270136.6
ENSG00000255054.3
ENSG00000254553.1
ENSG00000271741.1
ENSG00000283580.4
ENSG00000284989.1
ENSG00000288208.1
ENSG00000285839.1
ENSG00000256407.2
ENSG00000271723.5
ENSG00000284686.1
ENSG00000271949.1
ENSG00000289565.1
ENSG00000285779.1
ENSG00000285641.1
ENSG00000160818.17
ENSG00000196266.6
ENSG00000249730.1
ENSG00000256029.6
ENSG00000270149.5
ENSG00000244682.8
ENSG00000285777.1
ENSG00000286231.1
ENSG00000288674.1
ENSG00000270106.6
ENSG00000135747.11
ENSG00000197617.9
ENSG00000228336.2
ENSG00000227152.6
ENSG00000286905.1
ENSG00000276087.2
ENSG00000285542.1
ENSG00000273269.3
ENSG00000279956.1
ENSG00000273398.6
ENSG00000204872.4
ENSG00000264324.1
ENSG00000159239.13
ENSG00000274049.4
ENSG00000289685.2
ENSG00000241962.9
ENSG00000284337.1
ENSG00000257207.5
ENSG00000283228.1
ENSG00000280537.2
ENSG00000284820.1
ENSG00000286239.1
ENSG00000284622.2
ENSG00000258984.5
ENSG00000288550.3
ENSG00000272410.5
ENSG00000268279.4
ENSG00000293553.1
ENSG00

examining above cases where genes showing `gene_type` of `protein_coding` but doesn't have a transcript with `transcript_type` of `protein_coding`, these are genes marked as pc but the annotated transcripts are read through or LoF (e.g., ENSG00000285629.1, ENSG00000211454.16). I believe this is safe to ignore and just isolate the functional mRNA transcript sequences from GENCODE Basic (v27).

In [ ]:
pc_tids = []
out_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/gencode_pc_g2t_mapping.csv"
with open(out_fn, 'w') as fh:
    fh.write('gene_id,transcript_id\n')
    for x in pc_g2t:
        for y in pc_g2t[x]:
            pc_tids.append(y)
            fh.write(f'{x},{y}\n')

SANITY CHECK: are all Xenium target genes present in above set?

In [39]:
pc_q2t_keys = set([x.split('.')[0] for x in pc_g2t.keys()])
to_add = set()
for probe in probe_targets:
    gid, gname = probe_targets[probe]
    if gid not in pc_q2t_keys:
        to_add.add(gid)
to_add

{'ENSG00000184293', 'ENSG00000277734'}

There were 2 target genes with `gene_biotype` not set to protein coding. It's possible that there are additional genes that are protein coding but annotated as something else (e.g., `TR_C_gene`)
- (Alternative) we could throw in anything with a CDS annotated on it? will that include pseudogenes?

In [41]:
for pid in ginfos:
    if pid.split('.')[0] in to_add:
        for x in ginfos[pid]:
            pc_tids.append(x)

In [42]:
len(pc_tids) # inc by 2

64670

In [43]:
gencode_fa = pyfastx.Fasta("/ccb/salz4-3/hji20/off-target-probe-checker/data/gencode.v47.basic.annotation.fmted.fa")
out_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/gencode.v47.basic.annotation.fmted.pc_only.fa"
with open(out_fn, 'w') as fh:
    for ent in gencode_fa:
        if ent.name in pc_tids:
            fh.write(ent.raw)

Now, fix the probe sequence orientation:

In [93]:
out_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/xenium_human_breast_gepps.fwd_oriented.fa"
with open(out_fn, 'w') as fh:
    for ent in probes:
        if ent.name in to_rc:
            seq = Seq(ent.seq)
            out_s = seq.reverse_complement()
        else:
            out_s = ent.seq
        fh.write(f'>{ent.name}\n{out_s}\n')

In [94]:
out_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/xenium_human_breast_gepps.flipped.lst"
with open(out_fn, 'w') as fh:
    for qname in to_rc:
        fh.write(f'{qname}\n')

In [95]:
aln_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/results/bt2/xenium_human_breast_gepps.fwd_oriented.bam"
unmapped = []
with pysam.AlignmentFile(aln_fn, 'rb') as fh:
    for brec in fh:
        qname = brec.query_name
        if brec.is_unmapped: # highly likely to be intergenic
            unmapped.append(qname)
print(unmapped)

['ENSG00000127951|FGL2|e520e9b', 'ENSG00000127951|FGL2|8b24f03', 'ENSG00000127951|FGL2|759a825', 'ENSG00000127951|FGL2|dc67d31', 'ENSG00000127951|FGL2|73aadf1', 'ENSG00000198851|CD3E|02369ef', 'ENSG00000271503|CCL5|e28edbf', 'ENSG00000166428|PLD4|6696a0b', 'ENSG00000166428|PLD4|f5bb72b', 'ENSG00000166428|PLD4|0449862', 'ENSG00000166428|PLD4|937df62', 'ENSG00000166428|PLD4|9982460', 'ENSG00000166428|PLD4|d4621d0', 'ENSG00000166428|PLD4|3c5cd33', 'ENSG00000166428|PLD4|caa61e4', 'ENSG00000166428|PLD4|567ab93', 'ENSG00000166428|PLD4|34aae6b', 'ENSG00000166428|PLD4|a0daecf', 'ENSG00000166428|PLD4|6a3fa57', 'ENSG00000153563|CD8A|651fabe', 'ENSG00000153563|CD8A|9d95389', 'ENSG00000153563|CD8A|0710972', 'ENSG00000153563|CD8A|da9ac06', 'ENSG00000132170|PPARG|5d8a10b', 'ENSG00000196683|TOMM7|8205364', 'ENSG00000196683|TOMM7|d2ffb7c', 'ENSG00000196683|TOMM7|e89014c', 'ENSG00000196683|TOMM7|6b8d206', 'ENSG00000132170|PPARG|b660a48', 'ENSG00000169083|AR|a0c6719']


In [56]:
aln_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/results/bt2/xenium_human_breast_gene_expression_panel_probe_sequences.bam"
unmapped_2 = []
with pysam.AlignmentFile(aln_fn, 'rb') as fh:
    for brec in fh:
        qname = brec.query_name
        if brec.is_unmapped: # highly likely to be intergenic
            unmapped_2.append(qname)
print(unmapped_2)

['ENSG00000127951|FGL2|e520e9b', 'ENSG00000127951|FGL2|8b24f03', 'ENSG00000127951|FGL2|759a825', 'ENSG00000127951|FGL2|dc67d31', 'ENSG00000127951|FGL2|73aadf1', 'ENSG00000198851|CD3E|02369ef', 'ENSG00000271503|CCL5|e28edbf', 'ENSG00000166428|PLD4|6696a0b', 'ENSG00000166428|PLD4|f5bb72b', 'ENSG00000166428|PLD4|0449862', 'ENSG00000166428|PLD4|937df62', 'ENSG00000166428|PLD4|9982460', 'ENSG00000166428|PLD4|d4621d0', 'ENSG00000166428|PLD4|3c5cd33', 'ENSG00000166428|PLD4|caa61e4', 'ENSG00000166428|PLD4|567ab93', 'ENSG00000166428|PLD4|34aae6b', 'ENSG00000166428|PLD4|a0daecf', 'ENSG00000166428|PLD4|6a3fa57', 'ENSG00000153563|CD8A|651fabe', 'ENSG00000153563|CD8A|9d95389', 'ENSG00000153563|CD8A|0710972', 'ENSG00000153563|CD8A|da9ac06', 'ENSG00000132170|PPARG|5d8a10b', 'ENSG00000132170|PPARG|b660a48', 'ENSG00000196683|TOMM7|8205364', 'ENSG00000196683|TOMM7|d2ffb7c', 'ENSG00000196683|TOMM7|e89014c', 'ENSG00000196683|TOMM7|6b8d206', 'ENSG00000169083|AR|a0c6719']


In [99]:
print(f'{len(unmapped)}\t{len(unmapped_2)}')

30	30


In [97]:
for x in unmapped:
    if x not in unmapped_2:
        print(x)

In [57]:
out_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/xenium_human_breast_gepps.unmapped.fa"
with open(out_fn, 'w') as fh:
    for ent in probes:
        if ent.name in unmapped_2:
            fh.write(ent.raw)

In [62]:
aln_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/results/bt2/xenium_human_breast_gepps.pc_only.bam"
unmapped_3 = []
with pysam.AlignmentFile(aln_fn, 'rb') as fh:
    for brec in fh:
        if brec.is_unmapped:
            unmapped_3.append(brec.query_name)
print(len(unmapped_3))

35


In [63]:
for x in unmapped_3:
    if x not in unmapped:
        print(x)

ENSG00000184293|CLECL1|ad68b18
ENSG00000184293|CLECL1|b96bdcf
ENSG00000184293|CLECL1|04513e0
ENSG00000184293|CLECL1|c5726c8
ENSG00000117090|SLAMF1|135aee1


confirmed that the additional 5 comes from the previous list of probes that failed to map to their targets but produced an alignment to a lncRNA.